In [1]:
!pip install transformers torch flask pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.1 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.0-py3-none-any.whl size=21129 sha256=ac2122080370ce15cd57b75083535f3ea8ec94915d3f69ba162d9f5966868747
  Stored in directory: /root/.cache/pip/wheels/60/29/7b/f64332aa7e5e88fbd56d4002185ae22dcdc83b35b3d1c2cbf5
Successfully built pyngrok


In [2]:
from typing import List
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import os

tokenizer = AutoTokenizer.from_pretrained("juierror/flan-t5-text2sql-with-schema-v2")
model = AutoModelForSeq2SeqLM.from_pretrained("juierror/flan-t5-text2sql-with-schema-v2")

def get_prompt(table_name, attributes, question):
    prompt = f"""convert question into SQL query for table: {table_name} with attributes: {', '.join(attributes)}. question: {question}"""
    return prompt

def extract_attributes_from_csv(csv_file_path):
    # Read the CSV file using Pandas
    #df = pd.read_csv(csv_file_path)

    # Extract attribute names from the CSV file
    attributes = df.columns.tolist()

    return attributes, df

def prepare_input(question: str, table_name: str, attributes: List[str]):
    # Convert attribute names in the question to match the case in the CSV
    for attr in attributes:
        question = question.replace(attr.lower(), attr)

    # Replace "table_name" with "df"
    prompt = get_prompt("df", attributes, question)
    input_ids = tokenizer(prompt, max_length=512, return_tensors="pt").input_ids
    return input_ids

def inference(question: str, csv_file_path: str) -> str:
    attributes, df = extract_attributes_from_csv(csv_file_path)
    input_data = prepare_input(question=question, table_name="df", attributes=attributes)
    input_data = input_data.to(model.device)
    outputs = model.generate(inputs=input_data, num_beams=10, top_k=10, max_length=512)
    result = tokenizer.decode(token_ids=outputs[0], skip_special_tokens=True)
    return result

# Specify the path to your CSV file
#csv_file_path = 'train.csv'  # Replace with the actual path to your CSV file

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from io import StringIO

# Create a Flask app
app = Flask(__name__)

AT = "2W0o8sFpDYBUuPhsFaleZ7IL9xD_4VoNmhhCCR75C2xsMSjXc"               #Paste your Auth_token
ngrok.set_auth_token(AT)
port_no=5000
public_url =  ngrok.connect(port_no).public_url

# Define an API endpoint to receive questions and generate SQL queries
@app.route('/generate-sql', methods=['POST'])
def generate_sql():

    global user_question
    global uploaded_file
    global df
    global sql_query

    # Get the user_question from the request
    data = request.get_json()
    user_question = data.get('question')
    uploaded_file = data.get('csv_data')

    if user_question and uploaded_file:
        # Convert the CSV data string back to a DataFrame
        df = pd.read_csv(StringIO(uploaded_file))

    sql_query = inference(user_question, df)

    # Return the generated SQL as a response

    return jsonify({'sql_query': sql_query})

print(f"To acces the Global link please click {public_url}")

# Run the Flask app
if __name__ == '__main__':
    app.run()

To acces the Global link please click https://b2d2-35-236-140-18.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Oct/2023 11:34:10] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Oct/2023 11:34:11] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Oct/2023 11:35:51] "GET /generate-sql HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [20/Oct/2023 11:37:21] "GET /generate-sql HTTP/1.1" 405 -
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: 